In [1]:
# CHECK INSTALL
import sys
!{sys.executable} -m pip install charset-normalizer
!{sys.executable} -m pip install dataclasses
!{sys.executable} -m pip install dataclasses-serialization
!{sys.executable} -m pip install idna
!{sys.executable} -m pip install lxml
!{sys.executable} -m pip install more-properties
!{sys.executable} -m pip install mypy-extensions
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install orjson
!{sys.executable} -m pip install psutil
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install simpleeval
!{sys.executable} -m pip install toolz
!{sys.executable} -m pip install toposort
!{sys.executable} -m pip install typing-inspect
!{sys.executable} -m pip install typing_extensions
!{sys.executable} -m pip install urllib3
!{sys.executable} -m pip install websockets
!{sys.executable} -m pip install numpyencoder
!{sys.executable} -m pip install xlwings

In [2]:
# GET THE CONFIG
import utils
user1 = utils.getenvcached("QUETZALCOATL_USER1")
token1 = utils.getenvcached("QUETZALCOATL_TOKEN1")

In [3]:
from LaplaceAPIClient import *
import os

config_solver01_2000_13 = {
    "backend": "QUBOSimulatorBackend",
    "nshots": 2000,      
}

config_solver01_fujitsuda_13 = {
    "backend": "FujitsuDASimulatorBackend",
    "nshots": 1,
    "number_iterations": 1000,   #  = 1000,               # total number of itrations per run
    "number_runs": 16,           #  = 16,                  # number of stochastically independant runs
    "temperature_start": 5000,   # = 5000,               # start temperature for annealing as float value
    "temperature_end": 10,       # = 10,                 # end temperature for annealing as float value 
    "temperature_mode": 0,      # = 0,                         # 0: reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                                      # 1: reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                                      # 2: reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    "temperature_decay": 0.0095, # = 0.0095,             # see temperature_mode 0
    "temperature_interval": 1,   # = 1,                  # see temperature_mode 0
    "offset_increase_rate": 5000.0, # = 5000.0,             # increase of dynamic offset when no bit selected, set to 0.0 to switch off dynamic offset
    "rescale": 100.0
}

config_solver01_VQE_13 = {
    "backend": "VQESimulatorBackend",
    "ninit": 1,
    "nshots": 5,     
}

config_solver01_Optic_13 = {
    "backend": "OpticQUBOBackend",
    "nshots": 1,
}

config_solver01_Julia_13 = {
    "backend": "JuliaQUBOBackend",
    "nshots": 1   
}

We will solve: $ min_{x \ {0,1}^n }  x^t . Q. t $

In [4]:
qubo13q = QUBO(np.array([
    [0.0, -1.0,  4.0],
    [-1.0, 5.0, -9.0],
    [4.0, -9.0, -5.0]
]))

In [5]:

qubo13 = QUBOSolverProblem(qubo13q, config_solver01_2000_13, UserTokenSerde(user=user1, token=token1)) # type: ignore

def queryDone13(res: QUBOSolverSolution) -> None:
    print(res)
    if not "bestBinaryVector" in res.Results:
        raise Exception('failure')
    if not len(res.Results["bestBinaryVector"]) == 3:
        raise Exception('failure')
    if not abs(res.Results["bestBinaryVector"][0] - 0.0) < 1e-12:
        raise Exception('failure')
    if not abs(res.Results["bestBinaryVector"][1] - 1.0) < 1e-12:
        raise Exception('failure')
    if not abs(res.Results["bestBinaryVector"][2] - 1.0) < 1e-12:
        raise Exception('failure')

pqueryQUBOSolverProblem("https://api2.anzaetek.com:443/execute", qubo13, queryDone13)


{'Results': {'bestBinaryVector': [0.0, 1.0, 1.0]}, 'Status': {'credit': 100, 'message': 'done'}}
{"Results": {"bestBinaryVector": [0.0, 1.0, 1.0]}, "Status": {"credit": 100, "message": "done"}}
result=
QUBOSolverSolution(Results={'bestBinaryVector': array([0., 1., 1.])}, Status=OperationStatusSerde(credit=100, message='done'))


In [6]:
print("vqe")
qubo13_vqe = QUBOSolverProblem(qubo13q, config_solver01_VQE_13, UserTokenSerde(user=user1, token=token1)) # type: ignore
pqueryQUBOSolverProblem("https://api2.anzaetek.com:443/execute", qubo13_vqe, queryDone13)
print("vqe done")

vqe
{'Results': {'bestBinaryVector': [0.0, 1.0, 1.0]}, 'Status': {'credit': 100, 'message': 'done'}}
{"Results": {"bestBinaryVector": [0.0, 1.0, 1.0]}, "Status": {"credit": 100, "message": "done"}}
result=
QUBOSolverSolution(Results={'bestBinaryVector': array([0., 1., 1.])}, Status=OperationStatusSerde(credit=100, message='done'))
vqe done


In [7]:
print("fujitsu (SDK only)")
qubo13_fujitsu = QUBOSolverProblem(qubo13q, config_solver01_fujitsuda_13, UserTokenSerde(user=user1, token=token1)) # type: ignore
pqueryQUBOSolverProblem("https://api2.anzaetek.com:443/execute", qubo13_fujitsu, queryDone13)
print("fujitsu done")

fujitsu (SDK only)
{'Results': {'bestBinaryVector': [0.0, 1.0, 1.0]}, 'Status': {'credit': 100, 'message': 'done'}}
{"Results": {"bestBinaryVector": [0.0, 1.0, 1.0]}, "Status": {"credit": 100, "message": "done"}}
result=
QUBOSolverSolution(Results={'bestBinaryVector': array([0., 1., 1.])}, Status=OperationStatusSerde(credit=100, message='done'))
fujitsu done


In [8]:
print("julia")
qubo13_julia = QUBOSolverProblem(qubo13q, config_solver01_Julia_13, UserTokenSerde(user=user1, token=token1)) # type: ignore
pqueryQUBOSolverProblem("https://api2.anzaetek.com:443/execute", qubo13_julia, queryDone13)
print("julia done")

julia
{'Results': {'bestBinaryVector': [0.0, 1.0, 1.0]}, 'Status': {'credit': 100, 'message': 'done'}}
{"Results": {"bestBinaryVector": [0.0, 1.0, 1.0]}, "Status": {"credit": 100, "message": "done"}}
result=
QUBOSolverSolution(Results={'bestBinaryVector': array([0., 1., 1.])}, Status=OperationStatusSerde(credit=100, message='done'))
julia done


In [9]:
print("optic")
qubo13_optic = QUBOSolverProblem(qubo13q, config_solver01_Optic_13, UserTokenSerde(user=user1, token=token1)) # type: ignore
pqueryQUBOSolverProblem("https://api2.anzaetek.com:443/execute", qubo13_optic, queryDone13)
print("optic done")

optic
{'Results': {'bestBinaryVector': [0.0, 1.0, 1.0]}, 'Status': {'credit': 100, 'message': 'done'}}
{"Results": {"bestBinaryVector": [0.0, 1.0, 1.0]}, "Status": {"credit": 100, "message": "done"}}
result=
QUBOSolverSolution(Results={'bestBinaryVector': array([0., 1., 1.])}, Status=OperationStatusSerde(credit=100, message='done'))
optic done
